In [4]:
###create a folder or env named etl
## add __init__.py (from etl import extract,transform,load)
##Extract
import pandas as pd
from pathlib import Path

def consolidate_csv():
    
    new_folder =Path("extracts/new")
    csv_files =[item for item in new_folder.glob("*.csv")]
    
    if csv_files:
        dfs = []
        
        for csv_file in csv_files:
            dfs.append(pd.read_csv(csv_file))
        
        staging_table = pd.concat(dfs,ignore_index= True)
        staging_table['Order Date'] = pd.to_datetime(['Order date'])
        return staging_table
    else:
        print("There are no files in folder.")

In [2]:
import pandas as pd

def transform(staging_table: pd.DataFrame):
    # Create a copy of the staging table
    transformed_table = staging_table.copy()

    # Rename columns
    transformed_table.rename(columns={
        'order_date': 'order_date',
        'address': 'address'
    }, inplace=True)

    # Extract yy, mm, dd from the order date
    transformed_table['order_year'] = pd.to_datetime(transformed_table['order_date']).dt.year
    transformed_table['order_month'] = pd.to_datetime(transformed_table['order_date']).dt.month
    transformed_table['order_day'] = pd.to_datetime(transformed_table['order_date']).dt.day

    # Extract street, city, state, zip from the address
    transformed_table[['street', 'city', 'state', 'zip']] = transformed_table['address'].str.split(',', expand=True)

    # Rearrange columns
    transformed_table = transformed_table[['order_date', 'order_year', 'order_month', 'order_day', 'street', 'city', 'state', 'zip']]

    # Remove duplicates
    transformed_table.drop_duplicates(inplace=True)

    return transformed_table

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [3]:
#load
# Import required packages
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine,Engine
from sqlalchemy.engine import URL


@task
def get_engine():
    # Load environment variables
    env = load_dotenv)

    #Get values from env file
    HOST = os.getenv("HOST")
    DB = os.getenv("DB")
    USER = os.getenv("USER")
    PASS = os.getenv("PASS")

    #Define connection string
    connection_string + URL.CREATE(
        drivername = "mssql+pyodbc",
        host=HOST,
        database=DB,
        username=USER,
        password=PASS,
        query={"driver":"ODBC Driver 17 for SQL Server"}
    )
    
    #Create Engine
    engine = create_engine(connection_string)
    
    return engine


def load_to_db(transformed_table:pd.DataFrame,table_name:str):
    """ Load transfromed_table to database 
    """
    transformed_table.to_sql(table_name,con=get_engine(),if_exists="append",index=False)

SyntaxError: unmatched ')' (2350569705.py, line 12)

In [ ]:
#main pipeline outside etl package create a file named main.py
from etl import extract,transform,load

if __name__ =="__main__":
    
    staging_table = extract.consolidated_csv()
    transfromed_table = transform.transform(staging_table)
    load.load_to_db(transformed_table,"name of the table")
    
    ##USAGE
    
##in TERMINAL TYPE THIS python .\main.py